<a href="https://colab.research.google.com/github/Andrewsas/trabalho-fundamentos-cid/blob/master/Trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação de pacotes

In [11]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


#Importar pacotes


In [12]:
import pandas as pd
import string
import time
import csv
import random
from ast import literal_eval
from textblob import TextBlob
from google.colab import drive


#Baixar dataset

In [ ]:
#baixa o arquivo do dataset para o ambiente colab
!wget "http://tiagodemelo.info/datasets/dataset-v2.dat"



--2020-11-11 00:17:48--  http://tiagodemelo.info/datasets/dataset-v2.dat
Resolving tiagodemelo.info (tiagodemelo.info)... 108.167.188.189
Connecting to tiagodemelo.info (tiagodemelo.info)|108.167.188.189|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89773865 (86M)
Saving to: ‘dataset-v2.dat’

dataset-v2.dat      100%[===================>]  85.61M  20.3MB/s    in 4.7s    

2020-11-11 00:17:54 (18.1 MB/s) - ‘dataset-v2.dat’ saved [89773865/89773865]



In [ ]:
# ler o arquivo com dos dados do dataset baixado
file = open("./dataset-v2.dat",'r')

# sepera em linhas os dados
rows = file.readlines()

# fecha o arquivo
file.close()

# criar uma lista para salvar os dicionarios
dictionaries = []

for row in rows:
  #transforma literal em dicionario
  saveDic = literal_eval(row)

  #adiciona atributos de subdicionarios no dicionario mais externo
  saveDic.update(saveDic['itemReviewed'].items())
  saveDic.update(saveDic['itemReviewed']['address'].items())

  # #delete subdicionarios
  del saveDic['address']
  del saveDic['itemReviewed']

  #adiciona na lista de dicionarios
  dictionaries.append(saveDic)




In [ ]:
# transforma a lista de dicionario em dataframe
data = pd.DataFrame(dictionaries)

data.head()

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,ratingValue,name,website,email,phoneNumber,averageRating,streetAddress,addressLocality,addressRegion,postalCode,country
0,Jantar,692856288,2344932,larinhavercosa222,"Comida maravilhosa, lugar do verdadeiro chicle...",pt,True,"São Carlos, SP",26/07/2019,07/2019,5,Imperador dos Camarões,http://www.imperadordoscamaroes.com.br,contato@imperadordoscamaroes.com.br,+55 82 3231-4134,4.5,Avenida Doutor Antônio Gouveia,,Alagoas,57030-170,Brasil
1,Uma boa experiência.,229208487,3720906,Pano P,"Comida boa, drinks legais e um ambiente bem ac...",pt,True,"Manaus, AM",16/09/2014,None,4,Guacamole Natal,http://www.facebook.com/GuacamoleNatal,guacamolenatal@gmail.com,+55 84 3219-3646,4.5,Avenida Praia de Ponta Negra 9068 Ponta Negra,,Rio Grande do Norte,59092-100,Brasil
2,Otimo self service,679002332,2248105,Magma100,"Excelente pra ir com a família! Ótima Comida, ...",pt,True,"Goiânia, GO",04/06/2019,07/2018,4,Chica Pitanga,http://www.chicapitanga.com.br,chicapitanga@chicapitanga.com.br,+55 81 3334-8260,4.5,Rua Petrolina 19,Recife,Pernambuco,51021-250,Brasil
3,O sorvete mais tradicional do estado,355671519,8168302,Yurie M,Tomo sorvete na Visabor faz mais de 30 anos. S...,pt,False,"Aracaju, SE",15/03/2016,01/2016,5,Sorvetes Vi Sabor,None,None,None,4.0,Avenida Francisco Porto 59 Grageru,Aracaju,Sergipe,,Brasil
4,Melhor hamburger!,268741921,5395141,Debora_C85,O melhor hamburger que já comi! Excelente aten...,pt,False,"Rio de Janeiro, RJ",28/04/2015,04/2015,5,Z Deli Sandwich Shop,http://www.facebook.com/zdelisandwichshop/,None,551130830021,4.5,Rua Haddock Lobo 1386 Jardins,São Paulo,Estado de São Paulo,01414-002,Brasil


#Tratamento de Comentários Vazios

In [ ]:
#seleciona tuplas vazias
data[pd.isnull(data.reviewBody)]

,title,reviewId,restaurantId,author,reviewBody,language,mobileDevice,userLocation,publicationDate,visitDate,ratingValue,name,website,email,phoneNumber,averageRating,streetAddress,addressLocality,addressRegion,postalCode,country
8528,Vale a pena conhecer!,224445539,NaN,NaN,NaN,pt,False,"Recife, PE",25/08/2014,12/2013,4,Outback Steakhouse - Shopping RioMar Recife,https://www.outback.com.br/restaurantes/shoppi...,faleconosco@outback.com.br,+55 81 3035-0930,4.5,"Av República do Líbano, 256 Shopping RioMar Re...",Recife,Pernambuco,51110-160,Brasil


In [ ]:
data.reviewBody.fillna('VAZIO')

data.reviewBody[8].encode('utf-16', 'surrogatepass').decode('utf-16')

'Ambiente agradável, preço totalmente acessível, e adoro o cardápio de lá! Sem falar no atendimento maravilhoso! Voltarei sempre 😘'

#Obter sentenças

**Text Blob**

In [ ]:
# criar uma lista para salvar os blobs
comentario_blob = []

#Obter lista dos comentários
comentarios = data.reviewBody

# Iteração em cada review
for comentario in comentarios:
  #Obter sentenças tratando emojis
  aux = TextBlob(str(comentario).encode('utf-16', 'surrogatepass').decode('utf-16'))
  for sentence in aux.sentences:
    if sentence != "!" and sentence != "." and sentence != ".." and sentence != "rs" and sentence != "Aff." :
      comentario_blob.append(sentence)

In [ ]:
#Imprima as 10 primeiras sentenças
comentario_blob[:10]

[Sentence("Comida maravilhosa, lugar do verdadeiro chiclete de camarão."),
 Sentence("O garçom Éric Lima nos atendeu muito bem, pretendemos voltar assim que possível."),
 Sentence("Comida boa, drinks legais e um ambiente bem aconchegante."),
 Sentence("Provamos muitas coisas e tudo nos agradou."),
 Sentence("Principalmente um pato servido em um dos pratos da casa, sensacional!"),
 Sentence("Excelente pra ir com a família!"),
 Sentence("Ótima Comida, ambiente agradável, bom atendimento e estacionamento."),
 Sentence("Vale a pena conhecer"),
 Sentence("Tomo sorvete na Visabor faz mais de 30 anos."),
 Sentence("Sabores de frutas ou artificiais.")]

#Salvar no Drive

Montar drive

In [13]:
drive.mount('/content/drive/', force_remount='true')

Mounted at /content/drive/


Salvar no drive documento com comentários para tradução posterior. 

In [ ]:
!ls -la 'drive/My Drive/sentiment_analysis_project/'

drive_file = "drive/My Drive/sentiment_analysis_project/translate.csv" 
file = open(drive_file,'r')

data_translate = pd.read_csv(file)

# Iteração em cada review
for cmt in comentarios:
  try:
    #Obter sentenças tratando emojis
    comment_aux = str(cmt).encode('utf-16', 'surrogatepass').decode('utf-16')
    #cria objeto para adicionar ao dataframe
    obj_aux = { 'review': comment_aux, 'review_us': 'EMPTY' }
    # adiciona ao dataframe
    data_translate = data_translate.append(obj_aux, ignore_index=True)
  except:
    print("exception: " + cmt)

# Salva arquivo no google Drive
data_translate.to_csv(drive_file, index=False, header=True)    
data_translate

total 1
-rw------- 1 root root 18 Nov 11 01:16 translate.csv


,review,review_us
0,"Comida maravilhosa, lugar do verdadeiro chicle...",EMPTY
1,"Comida boa, drinks legais e um ambiente bem ac...",EMPTY
2,"Excelente pra ir com a família! Ótima Comida, ...",EMPTY
3,Tomo sorvete na Visabor faz mais de 30 anos. S...,EMPTY
4,O melhor hamburger que já comi! Excelente aten...,EMPTY
...,...,...
99995,"Ainda não inaugurado oficialmente, está atende...",EMPTY
99996,"Bom atendimento, lugar bonito, comida muito go...",EMPTY
99997,O Rio Scenarium é uma das baladas mais famosas...,EMPTY
99998,"Fui por indicação e com boa expectativa!\n116,...",EMPTY


#Traduzir sentenças

Comentários traduzidos para Inglês (US) a partir de CSV no google drive.

In [ ]:
drive_file = "drive/My Drive/sentiment_analysis_project/translate_us.csv" 
data_translate = pd.read_csv(drive_file)

dt = data_translate.loc[data_translate['review_us'] == 'EMPTY']

#Informação sobre quantos faltam
print(dt.describe())
#Obter lista dos comentários
comentarios = dt.review

# Iteração em cada review. Colocar o número de interações que deseja realizar.
# QTDE   | Minutos
# 500    | 4,5
# 1000   | 8,9
# 1500   | 13,51
# 3000   | 26,86
# 5000   | 44,73
for comentario in comentarios[:5000]:
  try:
    time.sleep(0.5)
    # Obter sentenças tratando emojis
    txt_blob_aux = TextBlob(str(comentario).encode('utf-16', 'surrogatepass').decode('utf-16'))
    aux_us = str(txt_blob_aux.translate(to='us'))
    print(aux_us)

    # Atualiza dataframe
    data_translate.at[(data_translate['review'] == comentario), 'review_us'] = aux_us
  except:
    print("EXCEPTION: " + str(txt_blob_aux))
    if txt_blob_aux.detect_language() == 'en':
      # Atualiza dataframe
      data_translate.at[(data_translate['review'] == comentario), 'review_us'] = str(txt_blob_aux)
      print("Up-to-date: " + str(txt_blob_aux))
      


# Salva arquivo no google Drive
data_translate.to_csv(drive_file, index=False, header=True)

data_translate

       review review_us
count       0         1
unique      0         1
top       NaN     EMPTY
freq      NaN         1
in


,review,review_us
0,"Comida maravilhosa, lugar do verdadeiro chicle...","Wonderful food, the place for real shrimp gum...."
1,"Comida boa, drinks legais e um ambiente bem ac...","Good food, nice drinks and a very cozy atmosph..."
2,"Excelente pra ir com a família! Ótima Comida, ...","Excellent to go with the family! Great food, p..."
3,Tomo sorvete na Visabor faz mais de 30 anos. S...,I have had ice cream at Visabor for over 30 ye...
4,O melhor hamburger que já comi! Excelente aten...,The best hamburger I've ever eaten! Excellent ...
...,...,...
99995,"Ainda não inaugurado oficialmente, está atende...","Not yet officially opened, it is serving in so..."
99996,"Bom atendimento, lugar bonito, comida muito go...","Good service, beautiful place, very tasty food..."
99997,O Rio Scenarium é uma das baladas mais famosas...,Rio Scenarium is one of the most famous ballad...
99998,"Fui por indicação e com boa expectativa!\n116,...",I went by indication and with good expectation...


#Gerar Senteças aleatórias

Obter dataset traduzido com textblob e gerar sentenças aleatórias para o ***ground-truth***/

In [22]:
# comentários com tradução
drive_file = "drive/My Drive/sentiment_analysis_project/translate_us.csv" 
data_random = pd.read_csv(drive_file)
# gerar novo dataset
data_sentencas = pd.DataFrame(columns= ['review', 'subjetividade', 'classificacao'])
# Tamanho do dataset de comentários em pt_br e us
size = len(data_random)
# auxiliares loop
max_sentences = 200
current = 0
array_pos = []

while current < max_sentences:
  i = random.randint(0, size)
  # Valida se a posição randômica já foi utilizada
  if i not in array_pos:
    #Obter sentenças tratando emojis
    comment_aux = data_random.iloc[i]
    txtb_review = TextBlob(str(comment_aux.review).encode('utf-16', 'surrogatepass').decode('utf-16'))
    
    # obter uma sentença aleatória do review sorteado
    sentence_br = random.choice(txtb_review.sentences)
    print('REVIEW: '+ str(txtb_review))
    print('SENTENÇA: '+ str(sentence_br))
    
    #cria objeto para adicionar ao dataframe
    if sentence_br != "!" and sentence_br != "." and sentence_br != ".." and sentence_br != "rs" and sentence_br != "Aff." :
      obj_aux = { 'review': sentence_br, 'subjetividade': '', 'classificacao': '' }
      # adiciona ao dataframe
      data_sentencas = data_sentencas.append(obj_aux, ignore_index=True)
      current += 1
    
  array_pos.append(i)
  print(array_pos)

# Salvar no drive
data_sentencas.to_csv('drive/My Drive/sentiment_analysis_project/ground_truth.csv', index=False)
data_sentencas



REVIEW: comida excelente, atendimento bom é perto do centro administrativo e de fácil acesso, ambiente climatizado 
SENTENÇA: comida excelente, atendimento bom é perto do centro administrativo e de fácil acesso, ambiente climatizado
[4477]
REVIEW: Ja viajei por mais de 33 paises, conheço mais de 200 cidades, mas o atendimento de Amaro do Camarada Riomar é o melhor do mundo, além das delícias do restaurante sem comparação, e ambiente extremamente agradável. Parabens Camarada!!
SENTENÇA: Parabens Camarada!
[4477, 67046]
REVIEW: Restaurante fantástico, ambiente aconchegante, comida maravilhosa com preço acessível e funcionários super educados
SENTENÇA: Restaurante fantástico, ambiente aconchegante, comida maravilhosa com preço acessível e funcionários super educados
[4477, 67046, 54955]
REVIEW: Como toda a rede, o Coco Bambu Salvador não poderia ser diferente...
Atendimento, qualidade e excelente custo benefício. Vale a pena ir ao restaurante para almoçar, jantar e até mesmo em datas come

,review,subjetividade,classificacao
0,"(c, o, m, i, d, a, , e, x, c, e, l, e, n, t, ...",,
1,"(P, a, r, a, b, e, n, s, , C, a, m, a, r, a, ...",,
2,"(R, e, s, t, a, u, r, a, n, t, e, , f, a, n, ...",,
3,"(O, , r, e, s, t, a, u, r, a, n, t, e, , p, ...",,
4,"(O, , p, r, e, ç, o, , c, o, b, r, a, d, o, ...",,
...,...,...,...
195,"(O, , r, e, s, t, a, u, r, a, n, t, e, , f, ...",,
196,"(L, o, c, a, l, i, z, a, ç, ã, o, , n, a, , ...",,
197,"(P, r, e, ç, o, s, , m, e, d, i, a, n, o, s, .)",,
198,"(M, e, l, h, o, r, , m, a, n, i, ç, o, b, a, ...",,
